### 모듈, 함수정의 <br>

In [1]:
import os
import sys
import struct
import numpy as np

In [2]:
# 파일 불러오기
def read_file(filename):
  contents = bytes()

  with open(filename, "rb") as f:
    f.seek(0, 2)
    num_points = int(f.tell() / 4)
    f.seek(0, 0)
    contents = f.read()

  arr = [struct.unpack('<I', contents[4 * i:4 * i + 4])[0] for i in range(num_points)]

  return arr

In [3]:
# 파일 저장하기
def save_file(filename, labels):
  arr = [struct.pack('<I', label) for label in labels]

  with open(filename, "bw") as f:
    for a in arr:
      f.write(a)

### 경로 지정, 파일 불러오기 <br>

In [4]:
os.getcwd()

'/home/alien4/바탕화면'

In [5]:
# 바탕화면에서 주피터노트북 시작하면 바탕화면 경로
# os.getcwd() 이용해도됨
cur_path = os.getcwd()

# Lane 라벨링 할 디렉토리명
work_dir = "LaneLabeling_A20"

# label,velodyne
labels_path = os.path.join(cur_path, work_dir, "labels")
velodyne_path = os.path.join(cur_path, work_dir, "velodyne")

In [6]:
# 파일경로 지정
for file in os.listdir(labels_path):
    if file.endswith(".label"):
        label_file = os.path.join(labels_path, file)
        
for file in os.listdir(velodyne_path):
    if file.endswith(".bin"):
        bin_file = os.path.join(velodyne_path, file)

In [7]:
#불러오기
labels=read_file(label_file)
bins=read_file(bin_file)

### bin 파일에서 주요값들 불러오기 <br>

In [8]:
# 점의 개수
num_points = len(labels)
print(num_points)

7853405


In [9]:
# 광도 정보들 brightness에 저장
brightness=[]
x=[]
y=[]
for i in range(num_points):
    x.append(bins[4*i])
    y.append(bins[4*i+1])
    brightness.append(bins[4*i+3])

### 파라미터 조절하기 <br>

In [10]:
# 빛의 세기 파라미터
def_intensity=1060000000 # default: intensity=1050000000

In [11]:
# 밝은 애들 모으기
num_high_intensity=[]
for i in range(num_points):
    if (brightness[i]>def_intensity) and (labels[i]==40):
        num_high_intensity.append(i)
len(num_high_intensity)

505717

### 로드만 적용할 수 있도록 조절 <br>

In [12]:
array_x = np.array(x)
array_y = np.array(y)

In [13]:
for i in range(num_points):
    if labels[i]!=40:
        array_x[i]=-1
        array_y[i]=-1

### Lane으로 바꾸기 <br>

In [14]:
# 빛의 세기 파라미터 배열
intensity=[] # default: intensity=1050000000
minus = 1000000

# 1060000000 ~ 1080000000
for i in range(21):
    intensity.append(def_intensity-i*minus+20000000)

In [15]:
num=0
cur_high_intensity=0
# intensity size 조정하고 돌리기
for size in intensity :
    
    num+=1
    # 밝은 애들 모으기
    high_intensity=[]
    for i in range(num_points):
        if (brightness[i]>size) and (labels[i]==40):
            high_intensity.append(i)
    
    cur_high_intensity += len(high_intensity)
    print(("작업량 {}: {}").format(num, len(high_intensity)))
            
    lane=[]
    for i in high_intensity:
        same_x = set(np.where(array_x==x[i])[0])
        same_y = set(np.where(array_y==y[i])[0])
        same = same_x.intersection(same_y)
        same = list(same)
        lane.append(same)
        
    lane_points=[]
    for i in range(len(lane)):
        for j in lane[i]:
            lane_points.append(j)
            
    # 보라색중에 밝은 보라색을 차선으로 바꾸기
    for i in lane_points:
        # 40=road , 60=lane
        labels[i]=60

print(("원래의 작업량 {} / 수정된 작업량 {} ").format(num_high_intensity,
                                       cur_high_intensity))

작업량 1: 356
작업량 2: 101
작업량 3: 118
작업량 4: 167
작업량 5: 299
작업량 6: 465
작업량 7: 1004
작업량 8: 1545
작업량 9: 3016
작업량 10: 6399
작업량 11: 11015
작업량 12: 16525


KeyboardInterrupt: 

In [51]:
#저장
save_file(label_file,labels)